In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os, sys, gc, random, multiprocessing, glob, time

#DATA_DIR = '../input/google-quest-challenge'
DATA_DIR = 'D:/project/ICF_AutoCapsule_disabled/kaggle/google-quest-challenge'
BERT_DIR = 'D:/project/ICF_AutoCapsule_disabled/BERT'

In [2]:
# !pip install ../input/sacremoses/sacremoses-master/
# !pip install ../input/transformers/transformers-master/

In [3]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

#from ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.utils import shuffle

from scipy.stats import spearmanr

import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)

from tqdm import tqdm
print(transformers.__version__)

2.2.2


In [4]:
## Make results reproducible .Else noone will believe you .
import random

def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = sptr(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [5]:
class PipeLineConfig:
    def __init__(self, lr, warmup, epochs, patience, batch_size, seed, name, question_weight,answer_weight,fold,train):
        self.lr = lr
        self.warmup = warmup
        self.epochs = epochs
        self.patience = patience
        self.batch_size = batch_size
        self.seed = seed
        self.name = name
        self.question_weight = question_weight
        self.answer_weight =answer_weight
        self.fold = fold
        self.train = train

In [6]:
config = PipeLineConfig(lr=1e-5, \
                        warmup=0.01, \
                        epochs=6, \
                        patience=3, \
                        batch_size=8, \
                        seed=42, \
                        name='test', \
                        question_weight=0.8, \
                        answer_weight=0.2, \
                        fold=3, \
                        train=True
                       )

In [7]:
seed_everything(config.seed)

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [9]:
sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,...,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,...,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,...,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,...,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,...,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [10]:
target_columns = sub.columns.values[1:].tolist()
target_columns

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [11]:
train = pd.read_csv(f'{DATA_DIR}/train.csv')
train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [12]:
test = pd.read_csv(f'{DATA_DIR}/test.csv')
test.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


## Dataset

In [13]:
MAX_LEN = 512
#MAX_Q_LEN = 250
#MAX_A_LEN = 259
SEP_TOKEN_ID = 102 # bert-base-uncasedにおけるvocabの'[SEP']が、102番目という意味

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True):
        self.df = df
        self.train_mode = train_mode
        self.labeled = labeled
        self.tokenizer = BertTokenizer.from_pretrained(BERT_DIR+'/bert-base-uncased')
        #self.tokenizer = BertTokenizer.from_pretrained('../input/bert-base-uncased/')

    def __getitem__(self, index):
        """
        token_id列
        segment_id列
        label列
        """
        row = self.df.iloc[index]
        token_ids, seg_ids = self.get_token_ids(row)
        if self.labeled:
            labels = self.get_label(row)
            return token_ids, seg_ids, labels
        else:
            return token_ids, seg_ids

    def __len__(self):
        return len(self.df)


#     def select_tokens(self, tokens, max_num):
#         if len(tokens) <= max_num:
#             return tokens
#         if self.train_mode:
#             num_remove = len(tokens) - max_num
#             remove_start = random.randint(0, len(tokens)-num_remove-1)
#             return tokens[:remove_start] + tokens[remove_start + num_remove:]
#         else:
#             return tokens[:max_num//2] + tokens[-(max_num - max_num//2):]

    def trim_input(self, title, question, answer, max_sequence_length=MAX_LEN, 
                t_max_len=30, q_max_len=239, a_max_len=239):
        """
        title. question, answerそれぞれのセンテンスを、tokenizeする
        max_lengthに足りない分は、
        """
        t = self.tokenizer.tokenize(title)
        q = self.tokenizer.tokenize(question)
        a = self.tokenizer.tokenize(answer)

        t_len = len(t)
        q_len = len(q)
        a_len = len(a)

        if (t_len+q_len+a_len+4) > max_sequence_length:

            if t_max_len > t_len:
                """
                titleが短い場合、
                最大長に足りない長さを、半分ずつqとaに加える
                """
                t_new_len = t_len
                a_max_len = a_max_len + math.floor((t_max_len - t_len)/2) # 切り捨て
                q_max_len = q_max_len + math.ceil((t_max_len - t_len)/2) # 切り上げ
            else:
                """
                titleが長い場合、最大長で切る
                """
                t_new_len = t_max_len

            if a_max_len > a_len:
                """
                answerに加えても短い場合、
                最大長に足りない長さを、qに加える
                """
                a_new_len = a_len 
                q_new_len = q_max_len + (a_max_len - a_len)
            elif q_max_len > q_len:
                a_new_len = a_max_len + (q_max_len - q_len)
                q_new_len = q_len
            else:
                a_new_len = a_max_len
                q_new_len = q_max_len


            if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
                raise ValueError("New sequence length should be %d, but is %d" 
                                 % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))

            t = t[:t_new_len]
            q = q[:q_new_len]
            a = a[:a_new_len]

        return t, q, a
        
    def get_token_ids(self, row):
        t_tokens, q_tokens, a_tokens = self.trim_input(row.question_title, row.question_body, row.answer)
        
        # BERTの入力タイプに変換([CLS]と[SEP]をつないで、１つのsetentenceに)
        tokens = ['[CLS]'] + t_tokens + ['[SEP]'] + q_tokens + ['[SEP]'] + a_tokens + ['[SEP]']
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        if len(token_ids) < MAX_LEN:
            """0で後ろからpadding"""
            token_ids += [0] * (MAX_LEN - len(token_ids))
        ids = torch.tensor(token_ids)
        seg_ids = self.get_seg_ids(ids)  # segment_embを区別するindex
        return ids, seg_ids
    
    def get_seg_ids(self, ids):
        """
        いくつめの文かを区別するsegment_idを、各文字に振る
        """
        seg_ids = torch.zeros_like(ids) # [max_len]のtorch_tensor
        seg_idx = 0
        first_sep = True
        for i, e in enumerate(ids):
            seg_ids[i] = seg_idx
            if e == SEP_TOKEN_ID: # [SEP]の場合
                if first_sep:
                    first_sep = False
                else:
                    seg_idx = 1
        pad_idx = torch.nonzero(ids == 0)  # bert-base_uncasedのvocabで、[PAD]は0番目であるので、PADの部分のindexだけ抽出
        seg_ids[pad_idx] = 0

        return seg_ids

    def get_label(self, row):
        #print(row[target_columns].values)
        return torch.tensor(row[target_columns].values.astype(np.float32))

    def collate_fn(self, batch):
        """
        labelデータを持つモードと、ない完全な推論モードでは、batchのshapeが異なるので(labelが2番目の要素にあるなし)
        """
        token_ids = torch.stack([x[0] for x in batch])
        seg_ids = torch.stack([x[1] for x in batch])
    
        if self.labeled:
            labels = torch.stack([x[2] for x in batch])
            return token_ids, seg_ids, labels
        else:
            return token_ids, seg_ids

In [14]:
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0):
    df = pd.read_csv(f'{DATA_DIR}/train.csv')
    df = shuffle(df, random_state=1234)
    gkf = GroupKFold(n_splits=5).split(X=df.question_body, groups=df.question_body)
    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold == ifold:
            df_train = df.iloc[train_idx]
            df_val = df.iloc[valid_idx]
            break

    print('train', df_train.shape)
    print('val', df_val.shape)

    ds_train = QuestDataset(df_train, train_mode=True)
    train_loader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=ds_train.collate_fn, drop_last=True)
    train_loader.num = len(df_train)

    ds_val = QuestDataset(df_val, train_mode=False)
    val_loader = torch.utils.data.DataLoader(ds_val, batch_size=val_batch_size, shuffle=False, num_workers=0, collate_fn=ds_val.collate_fn, drop_last=False)
    val_loader.num = len(df_val)
    val_loader.df = df_val

    return train_loader, val_loader, df_val.shape[0]


def get_test_loader(batch_size=4):
    df = pd.read_csv(f'{DATA_DIR}/test.csv')
    df_test = QuestDataset(df, train_mode=False, labeled=False)
    loader = torch.utils.data.DataLoader(df_test, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=ds_test.collate_fn, drop_last=False)
    loader.num = len(df)
    
    return loader


# def test_train_loader():
#     loader, _ = get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=1)
#     for ids, seg_ids, labels in loader:
#         print(ids)
#         print(seg_ids.numpy())
#         print(labels)
#         break
# def test_test_loader():
#     loader = get_test_loader(4)
#     for ids, seg_ids in loader:
#         print(ids)
#         print(seg_ids)
#         break

In [15]:
class QuestModel(nn.Module):
    def __init__(self, n_classes=30):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
        self.bert_model = BertModel.from_pretrained(BERT_DIR+'/bert-base-uncased/')
        #self.bert_model = BertModel.from_pretrained('../input/bert-base-uncased/')    
        self.fc = nn.Linear(768, n_classes)

    def forward(self, ids, seg_ids):
        attention_mask = (ids > 0)  # ids==0([PAD])部分だけFalseとなるので、そこだけattention_weightを0に
        layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=seg_ids, attention_mask=attention_mask)
        #print(layers[-1][0].size())
        #print(pool_out.size())

        #out = F.dropout(layers[-1][:, 0, :], p=0.2, training=self.training)
        out =  F.dropout(pool_out, p=0.2, training=self.training)
        logit = self.fc(out)
        return logit # 単に30種類の出力値を算出
    
def test_model():
    """確認用、バッチサイズ2"""
    x = torch.tensor([[1,2,3,4,5,0, 0], [1,2,3,4,5, 0, 0]])
    seg_ids = torch.tensor([[0,0,0,0,0, 0, 0], [0,0,0,0,0, 0, 0]])
    model = QuestModel()

    y = model(x, seg_ids)
    print(y.shape)
    print(y)

In [16]:
test_model()

torch.Size([2, 30])
tensor([[ 3.9142e-01, -1.3947e-02,  5.2440e-01,  7.0391e-01,  3.6601e-02,
         -2.1076e-01, -2.0742e-01,  3.8842e-01, -3.6691e-01,  3.9539e-02,
          1.1247e-01, -1.7837e-01, -2.8799e-01,  1.8482e-01,  3.0477e-01,
         -4.5973e-01,  3.3056e-01, -1.4741e-01,  4.4749e-02,  2.1768e-01,
          4.1020e-01,  8.8369e-02,  3.2358e-01,  3.7427e-01,  3.9418e-01,
          5.1816e-04, -2.7123e-01,  4.2593e-01, -9.0014e-02,  9.4861e-01],
        [ 4.6783e-01, -3.0907e-02,  5.7824e-01,  4.3496e-01,  9.0965e-02,
         -2.3794e-01, -1.4856e-01,  1.3355e-01, -2.2786e-01,  3.1250e-01,
         -4.0960e-01, -2.1083e-01, -3.8161e-01, -6.3352e-02,  6.0217e-01,
         -5.3606e-01,  3.0912e-01, -1.1719e-02,  9.1177e-02,  3.0557e-01,
          9.0634e-01,  1.8659e-02,  2.4406e-01,  3.0113e-01,  4.8014e-01,
          2.4510e-01, -2.3487e-01,  5.5336e-01, -8.4923e-02,  5.2944e-01]],
       grad_fn=<AddmmBackward>)


In [17]:
def train_model(train_loader, optimizer, criterion, scheduler):
    model.train()
    avg_loss = 0.    
    for idx, batch in enumerate(train_loader):
        ids_train, seg_ids_train, label_ids_train = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        
        logits = model(ids_train, seg_ids_train)
        #logits = torch.sigmoid(model(ids_train, seg_ids_train))
        
        loss = config.question_weight*criterion(logits[:,0:21], label_ids_train[:,0:21]) + config.answer_weight*criterion(logits[:,21:30], label_ids_train[:,21:30])
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        del ids_train, seg_ids_train, label_ids_train

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

def val_model(val_loader, val_length, batch_size=8):
    model.eval() # eval mode  
    avg_val_loss = 0.
    
    valid_preds = np.zeros((val_length, len(target_columns)))
    original = np.zeros((val_length, len(target_columns)))
    
    with torch.no_grad():
        for idx, batch in enumerate(val_loader):
            ids_val, seg_ids_val, label_ids_val = batch[0].to(device), batch[1].to(device), batch[2].to(device)
            
            logits = torch.sigmoid(model(ids_val, seg_ids_val))
            
            avg_val_loss += criterion(logits, labels).item() / len(val_loader)
            valid_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
        
        score = 0
        preds = torch.tensor(valid_preds).numpy()
        #preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()
        
        # np.save("preds.npy", preds)
        # np.save("actuals.npy", original)
        
        rho_val = np.mean([spearmanr(original[:, i], preds[:,i]).correlation for i in range(preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        for i in range(len(target_columns)):
            print(i, spearmanr(original[:,i], preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], preds[:, i]).correlation)
    
    return avg_val_loss, score/len(target_columns)

In [18]:
train_loader, val_loader, val_length = get_train_val_loaders()
for n, d in enumerate(train_loader):
    if n ==0:
        print(len(d))
        print(d[0])
        print(d[1])
        print(d[2])
    else:
        break

train (4863, 41)
val (1216, 41)
3
tensor([[  101,  9765, 22835,  ..., 19761,  3064,   102],
        [  101,  2339,  2024,  ...,     0,     0,     0],
        [  101, 19998,  7367,  ...,  1013,  1007,   102],
        [  101,  1000,  8112,  ...,     0,     0,     0]])
tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[1.0000, 0.4444, 0.0000, 0.5000, 1.0000, 0.5000, 0.4444, 0.4444, 0.3333,
         0.0000, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.3333, 0.3333,
         0.0000, 0.0000, 0.8889, 0.7778, 0.7778, 0.7778, 0.7778, 0.7333, 0.3333,
         0.6667, 0.3333, 0.8889],
        [1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000, 0.6667, 0.5556, 0.0000,
         0.0000, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000, 1.0000, 1.0000, 0.5556, 1.0000, 0.8889, 0.7333, 0.0000,
         0.0000, 1.0000, 1.0000],
        [0.7778, 0.3333, 0.3333, 0.5000, 0

In [19]:
ACCUM_STEPS = 1

In [20]:
model = QuestModel(n_classes=30).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, eps=4e-5)
criterion = nn.BCEWithLogitsLoss()

In [21]:
for fold in range(config.fold):
    print('---%d-Fold---'%(fold+1))
    
    patience = 0
    best_loss   = 100.0
    best_score      = -1.
    best_param_loss = None
    best_param_score = None
    
    for epoch in tqdm(range(config.epochs)):
        
        torch.cuda.empty_cache()
        start_time   = time.time()
        
        train_loader, val_loader, val_length = get_train_val_loaders(batch_size=config.batch_size, val_batch_size=config.batch_size, ifold=fold)
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup, num_training_steps= config.epochs*len(train_loader)//ACCUM_STEPS)
        
        loss_train = train_model(train_loader, optimizer, criterion, scheduler)
        loss_val, score_val = val_model(val_loader, val_length, batch_size=config.batch_size)
        print(f'Epoch {(epoch+1)}, train_loss: {loss_train}, val_loss: {loss_val}, score_val: {score_val}, time: {(time.time()-start_time)}')
        
#         if loss_val < best_loss:
#             patience = 0
#             best_loss = loss_val 
#             best_param_loss = model.state_dict()

        if score_val > best_score:
            best_score = score_val
            best_param_score = model.state_dict()
            print('best_param_score_{}_{}.pt'.format(config.name ,fold+1))
            torch.save(best_param_score, 'best_param_score_{}_{}.pt'.format(config.name ,fold+1))
        else:
            patience += 1
            if patience >= config.patience:
                break
        
        del train_loader, val_loader, loss_train, loss_val, score_val
        torch.cuda.empty_cache()
    
    model.load_state_dict(best_param_score)
    print('best_param_score_{}_{}.pt'.format(config.name ,fold+1))
    torch.save(best_param_score, 'best_param_score_{}_{}.pt'.format(config.name ,fold+1))   
    
    del train_loader, val_loader, loss_train, loss_val, score_val
    torch.cuda.empty_cache()
    gc.collect()

---1-Fold---


  0%|                                                                         | 0/6 [00:00<?, ?it/s]

train (4863, 41)
val (1216, 41)


  0%|                                                                         | 0/6 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 4.00 GiB total capacity; 805.93 MiB already allocated; 23.77 MiB free; 38.07 MiB cached)

In [ ]:
def create_model(model_file):
    model = QuestModel()
    model.load_state_dict(torch.load(model_file))
    model = model.cuda()
    #model = DataParallel(model)
    return model

def create_models():
    models = []
    for fold in range(config.fold):
        model = create_model(f'../input/models/best_param_score_{}_{}.pt'.format(config.name ,fold+1))
        model.eval()
        models.append(model)
    return models

In [ ]:
def predict(models, test_loader):
    all_scores = []
    with torch.no_grad():
        for ids, seg_ids in tqdm(test_loader, total=test_loader.num // test_loader.batch_size):
            ids, seg_ids = ids.cuda(), seg_ids.cuda()
            scores = []
            for model in models:
                outputs = torch.sigmoid(model(ids, seg_ids)).cpu()
                scores.append(outputs)
            all_scores.append(torch.mean(torch.stack(scores), 0))

    all_scores = torch.cat(all_scores, 0).numpy()
    
    return all_scores

In [ ]:
test_loader = get_test_loader(batch_size=32)
models = create_models()

preds = predict(models, test_loader)

In [ ]:
sub[target_columns] = preds
sub.to_csv('submission.csv', index=False)
sub.head()

## 過去ログ

In [ ]:
class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, lengths, labels = None):
        
        self.inputs = inputs
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None
        self.lengths = lengths

    def __getitem__(self, idx):
        
        input_ids       = self.inputs[0][idx]
        input_masks     = self.inputs[1][idx]
        input_segments  = self.inputs[2][idx]
        lengths         = self.lengths[idx]
        if self.labels is not None: # targets
            labels = self.labels[idx]
            return input_ids, input_masks, input_segments, labels, lengths
        return input_ids, input_masks, input_segments, lengths

    def __len__(self):
        return len(self.inputs[0])

In [ ]:
def train_model(train_loader, optimizer, criterion, scheduler,config):
    
    model.train()
    avg_loss = 0.
   # tk0 = tqdm(enumerate(train_loader),total =len(train_loader))
    
    for idx, batch in enumerate(train_loader):
        
        input_ids, input_masks, input_segments, labels, _ = batch
        input_ids, input_masks, input_segments, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), labels.to(device)            
        
        output_train = model(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                            )
        logits = output_train[0] #output preds

        loss = config.question_weight*criterion(logits[:,0:21], labels[:,0:21]) + config.answer_weight*criterion(logits[:,21:30], labels[:,21:30])
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        del input_ids, input_masks, input_segments, labels

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

def val_model(val_loader, val_shape, batch_size=8):

    avg_val_loss = 0.
    model.eval() # eval mode
    
    valid_preds = np.zeros((val_shape, len(target_cols)))
    original = np.zeros((val_shape, len(target_cols)))
    
    #tk0 = tqdm(enumerate(val_loader))
    with torch.no_grad():
        
        for idx, batch in enumerate(val_loader):
            input_ids, input_masks, input_segments, labels, _ = batch
            input_ids, input_masks, input_segments, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), labels.to(device)            
            
            output_val = model(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                            )
            logits = output_val[0] #output preds
            
            avg_val_loss += criterion(logits, labels).item() / len(val_loader)
            valid_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
        
        score = 0
        preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()
        
        # np.save("preds.npy", preds)
        # np.save("actuals.npy", original)
        
        rho_val = np.mean([spearmanr(original[:, i], preds[:,i]).correlation for i in range(preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        for i in range(len(target_cols)):
            print(i, spearmanr(original[:,i], preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], preds[:, i]).correlation)
    return avg_val_loss, score/len(target_cols)

In [ ]:
def predict_result(model, test_loader, batch_size=32):
    
    test_preds = np.zeros((len(test), len(target_cols)))
    
    model.eval();
    tk0 = tqdm(enumerate(test_loader))
    for idx, x_batch in tk0:
        with torch.no_grad():
            outputs = model(input_ids = x_batch[0].to(device), 
                            labels = None, 
                            attention_mask = x_batch[1].to(device),
                            token_type_ids = x_batch[2].to(device),
                           )
            predictions = outputs[0]
            test_preds[idx*batch_size : (idx+1)*batch_size] = predictions.detach().cpu().squeeze().numpy()

    output = torch.sigmoid(torch.tensor(test_preds)).numpy()        
    return output

In [ ]:
tokenizer = BertTokenizer.from_pretrained("D:/project/ICF_AutoCapsule_disabled/BERT/bert-base-uncased/vocab.txt", do_lower_case=True)
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True, proxies={'https://s3.amazonaws.com':'http://proxygate2.nic.nec.co.jp:8080'})
input_categories = list(train.columns[[1,2,5]]); input_categories

bert_model_config = 'D:/project/ICF_AutoCapsule_disabled/BERT/bert-base-uncased/bert_config.json'
bert_config = BertConfig.from_json_file(bert_model_config)
bert_config.num_labels = len(target_columns)


bert_model = 'bert-base-cased'
do_lower_case = 'uncased' in bert_model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
output_model_file = 'bert_pytorch.bin'

from sklearn.model_selection import GroupKFold

test_inputs = compute_input_arays(test, input_categories, tokenizer, max_sequence_length=512)
lengths_test = np.argmax(test_inputs[0] == 0, axis=1)
lengths_test[lengths_test == 0] = test_inputs[0].shape[1]

print(do_lower_case, bert_model, device, output_model_file)

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


# In[8]:

NUM_FOLDS = config.fold  # change this
SEED = config.seed
BATCH_SIZE = 8
epochs = config.epochs   # change this
ACCUM_STEPS = 1

kf = KFold(n_splits = NUM_FOLDS, random_state = SEED)

test_set = QuestDataset(inputs=test_inputs, lengths=lengths_test, labels=None)
test_loader  = DataLoader(test_set, batch_size=32, shuffle=False)
result = np.zeros((len(test), len(target_columns)))

y_train = train[target_columns].values # dummy

print(bcolors.FAIL, f"For Every Fold, Train {epochs} Epochs", bcolors.ENDC)


if config.train :
    for fold, (train_index, val_index) in enumerate(kf.split(train.values, y_train)):

        print(bcolors.HEADER, "Current Fold:", fold, bcolors.ENDC)

        train_df, val_df = train.iloc[train_index], train.iloc[val_index]
        print("Train and Valid Shapes are", train_df.shape, val_df.shape)
    
        print(bcolors.HEADER, "Preparing train datasets....", bcolors.ENDC)
    
        inputs_train = compute_input_arays(train_df, input_categories, tokenizer, max_sequence_length=512)
        outputs_train = compute_output_arrays(train_df, columns = target_columns)
        outputs_train = torch.tensor(outputs_train, dtype=torch.float32)
        lengths_train = np.argmax(inputs_train[0] == 0, axis=1)
        lengths_train[lengths_train == 0] = inputs_train[0].shape[1]
    
        print(bcolors.HEADER, "Preparing Valid datasets....", bcolors.ENDC)
    
        inputs_valid = compute_input_arays(val_df, input_categories, tokenizer, max_sequence_length=512)
        outputs_valid = compute_output_arrays(val_df, columns = target_columns)
        outputs_valid = torch.tensor(outputs_valid, dtype=torch.float32)
        lengths_valid = np.argmax(inputs_valid[0] == 0, axis=1)
        lengths_valid[lengths_valid == 0] = inputs_valid[0].shape[1]
    
        print(bcolors.HEADER, "Preparing Dataloaders Datasets....", bcolors.ENDC)

        train_set    = QuestDataset(inputs=inputs_train, lengths=lengths_train, labels=outputs_train)
        train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
    
        valid_set    = QuestDataset(inputs=inputs_valid, lengths=lengths_valid, labels=outputs_valid)
        valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
    
        model = BertForSequenceClassification.from_pretrained('D:/project/ICF_AutoCapsule_disabled/BERT/bert-base-uncased/', config=bert_config);
        model.zero_grad();
        model.to(device);
        torch.cuda.empty_cache()
        model.train();
    
        i = 0
        best_avg_loss   = 100.0
        best_score      = -1.
        best_param_loss = None
        best_param_score = None

        optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, eps=4e-5)
        criterion = nn.BCEWithLogitsLoss()
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup, num_training_steps= epochs*len(train_loader)//ACCUM_STEPS)
        print("Training....")
    
        for epoch in range(config.epochs):
        
            torch.cuda.empty_cache()
        
            start_time   = time.time()
            avg_loss     = train_model(train_loader, optimizer, criterion, scheduler,config)
            avg_val_loss, score = val_model(valid_loader, val_shape=val_df.shape[0])
            elapsed_time = time.time() - start_time

            print(bcolors.OKGREEN, 'Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t score={:.6f} \t time={:.2f}s'.format(
                epoch + 1, epochs, avg_loss, avg_val_loss, score, elapsed_time),
            bcolors.ENDC
            )

            if best_avg_loss > avg_val_loss:
                i = 0
                best_avg_loss = avg_val_loss 
                best_param_loss = model.state_dict()

            if best_score < score:
                best_score = score
                best_param_score = model.state_dict()
                print('best_param_score_{}_{}.pt'.format(config.expname ,fold+1))
                torch.save(best_param_score, DATA_DIR+'best_param_score_{}_{}.pt'.format(config.expname ,fold+1))
            else:
                i += 1

            
        model.load_state_dict(best_param_score)
        result += predict_result(model, test_loader)
        print('best_param_score_{}_{}.pt'.format(config.expname ,fold+1))
        torch.save(best_param_score, DATA_DIR+'best_param_score_{}_{}.pt'.format(config.expname ,fold+1))
        
        result /= NUM_FOLDS
        
    del train_df, val_df, model, optimizer, criterion, scheduler
    torch.cuda.empty_cache()
    del valid_loader, train_loader, valid_set, train_set
    torch.cuda.empty_cache()
    torch.cuda.empty_cache()
    torch.cuda.empty_cache()
    gc.collect()

    
print(result)

In [ ]:
## Loading the colab pretrained model , same setting as experiement 1 for uncased . 
## I had to break and load as dataset . Thats why this weird way of loading weights . 
result = np.zeros((len(test), len(target_columns)))
#good_folds = [2,3,4,6,8]## Taking only the folds for which the val_spearman was more than 0.381 
for i in range(1,9):
    model = BertForSequenceClassification.from_pretrained('D:/project/ICF_AutoCapsule_disabled/BERT/bert-base-uncased/', config=bert_config);
    model.zero_grad();
    model.to(device);
    if i < 5 :
        model.load_state_dict(torch.load(DATA_DIR+'/best_param_score_uncased_1_{i}.pt'))
    else:
        model.load_state_dict(torch.load(DATA_DIR+'/best_param_score_uncased_1_{i}.pt'))
        
    result += predict_result(model, test_loader)
    
for i in range(1,5):
    model = BertForSequenceClassification.from_pretrained('D:/project/ICF_AutoCapsule_disabled/BERT/bert-base-uncased/', config=bert_config);
    model.zero_grad();
    model.to(device);
    model.load_state_dict(torch.load(DATA_DIR+'/best_param_score_uncased_4_{i}.pt'))

    result += predict_result(model, test_loader)   
    
    
result/=12 ##  12 folds used out of 12 folds  